In [1]:
# !pip install pulp

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join, basename
from glob import glob
import re
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, value

In [3]:
current_season = 2022

id_columns = ["first_name", "last_name", "dob"]

In [4]:
# import and merge all spreadsheets based on the id columns
search_pattern = join(".", "table", "players_*.xlsx")

filepaths = glob(search_pattern)

seasons = []
players_past = None
for fp in filepaths:
    
    season = int(re.findall("players_(\d+).xlsx", basename(fp))[0])
    seasons.append(season)
    
    if season == current_season:
        players_current = pd.read_excel(fp)
        players_current = players_current[id_columns + ["club", "position", "price"]]
    else:
        players = pd.read_excel(fp)
        players = players[id_columns + ["points"]]   
        players["points_" + str(season)] = players.pop("points")
        
        if players_past is None:
            players_past = players.copy()
        else:
            players_past = pd.merge(players_past, players, 
                               how="outer", 
                               on=id_columns)
            
players = pd.merge(players_past, players_current, 
                               how="right", 
                               on=id_columns)

players.tail(n=5)

players.to_csv("players_2019_to_2021.csv", index=False, na_rep="nan")

In [5]:
# compute correlation of past seasons depending on time difference in years
corr = {}

for season_1 in seasons:
    for season_2 in seasons:
        if (season_1 != current_season) and (season_2 != current_season) and (season_2 > season_1):
            dt = season_2 - season_1
            
            if dt not in corr:
                corr[dt] = []
                
            print(players[f"points_{season_1:d}"].corr(players[f"points_{season_2:d}"]))
            corr[dt].append(players[f"points_{season_1:d}"].corr(players[f"points_{season_2:d}"]))
            
dt_max = 0
for dt in corr:
    corr[dt] = np.mean(corr[dt])
    
    if dt > dt_max:
        dt_max = dt

corr

0.6076910237912831
0.3774484706352427
0.5368446395943336


{1: 0.5722678316928084, 2: 0.3774484706352427}

In [6]:
# predict the correlation for an additional year assuming exponential decay
if dt_max > 1:
    dcorr = []
    for dt in range(2, dt_max + 1):
        dcorr.append(corr[dt] / corr[dt - 1])
        
    dcorr = np.mean(dcorr)
else:
    print("WARNING: Not enough seasons to estimate change of correlation. Defaulting to dcorr = 0.5")
    dcorr = 0.5
    
corr[dt_max + 1] = corr[dt_max] * dcorr

corr

{1: 0.5722678316928084, 2: 0.3774484706352427, 3: 0.24895222148596974}

In [7]:
# normalize the correlations to sum to 1
factor = 1 / sum(corr.values())
weight = {key: value * factor for key, value in corr.items() }

weight

{1: 0.477419587086445, 2: 0.314889782401432, 3: 0.20769063051212305}

In [8]:
# give 0 points for unrated seasons
players = players.fillna(0)

players.tail(n=5)

,first_name,last_name,dob,points_2019,points_2020,points_2021,club,position,price
548,stevan,jovetic,02/11/1989,0.0,0.0,92.0,hertha bsc,attack,2.2
549,timo,baumgartl,04/03/1996,0.0,0.0,124.0,1. fc union berlin,defense,2.2
550,tobias,strobl,12/05/1990,22.0,10.0,4.0,fc augsburg,midfield,1.0
551,wahid,faghir,29/07/2003,0.0,0.0,19.0,vfb stuttgart,attack,1.0
552,yussuf,poulsen,15/06/1994,66.0,79.0,121.0,rb leipzig,attack,3.0


In [9]:
# predict the points for the current season based on the correlations with past points
players[f"prediction"] = 0
for season in seasons:
    if season != current_season:
        players[f"prediction"] += weight[current_season - season] * players[f"points_{season:d}"]
        
players.tail(n=5)

,first_name,last_name,dob,points_2019,points_2020,points_2021,club,position,price,prediction
548,stevan,jovetic,02/11/1989,0.0,0.0,92.0,hertha bsc,attack,2.2,43.922602
549,timo,baumgartl,04/03/1996,0.0,0.0,124.0,1. fc union berlin,defense,2.2,59.200029
550,tobias,strobl,12/05/1990,22.0,10.0,4.0,fc augsburg,midfield,1.0,9.627770
551,wahid,faghir,29/07/2003,0.0,0.0,19.0,vfb stuttgart,attack,1.0,9.070972
552,yussuf,poulsen,15/06/1994,66.0,79.0,121.0,rb leipzig,attack,3.0,96.351644


In [13]:
# define the football manager league rules
formations = [
    (1, 3, 5, 2),
    (1, 4, 5, 1),
    (1, 4, 4, 2),
    (1, 4, 3, 3),
    (1, 3, 4, 3),
    (1, 5, 3, 2),
    (1, 5, 4, 1),
]

substitutes = [
    (1, 1, 0, 0),
    (1, 0, 0, 1),
    (1, 0, 1, 0),
    (1, 0, 1, 0),
    (1, 1, 0, 0),
    (1, 0, 1, 0),
    (1, 0, 0, 1),
]

# formations = [
#     (3, 6, 8, 5)
# ]
# substitutes = [
#     (0, 0, 0, 0)
# ]

n_same_club = 2
budget = 42.5
price_min = 0.5
n_player = 22

In [14]:
# define dictionaries linking player ids with attributes
price = dict(zip(players.index, players["price"]))
prediction = dict(zip(players.index, players["prediction"]))

positions = ["goal", "defense", "midfield", "attack"]
is_pos = {}
for pos in positions:
    is_pos[pos] = dict(zip(players.index, players["position"] == pos))
    
clubs = list(players["club"].unique())
is_club = {}
for c in clubs:
    is_club[c] = dict(zip(players.index, players["club"] == c))

In [15]:
# predict the best squad for each formation using binary linear programming
for form, sub in zip(formations, substitutes):
    lp_prob = LpProblem("team_selection", LpMaximize)

    player_vars = LpVariable.dicts("player", players.index, cat='Binary')

    budget_remain = budget - price_min * (n_player - sum(form) - sum(sub))
    
    lp_prob += lpSum([prediction[p_id] * player_vars[p_id] for p_id in players.index])
    lp_prob += lpSum([price[p_id] * player_vars[p_id] for p_id in players.index]) <= budget_remain
    for jj, pos in enumerate(positions):
        lp_prob += lpSum([is_pos[pos][p_id] * player_vars[p_id] for p_id in players.index]) == form[jj] + sub[jj]
    for c in clubs:
        lp_prob += lpSum([is_club[c][p_id] * player_vars[p_id] for p_id in players.index]) <= n_same_club               

    lp_prob.solve()
    
    print(form)
    for var in lp_prob.variables():
        
        player_binary = var.varValue
        if player_binary == 1:
            player_id = int(var.name.split("_")[1])

            player_name = " ".join(players.loc[players.index == player_id, ["first_name", "last_name"]].values[0])
            player_points =  players.loc[players.index == player_id, "prediction"].values[0]
            player_price =  players.loc[players.index == player_id, "price"].values[0]
            player_position =  players.loc[players.index == player_id, "position"].values[0]
            print(player_name + f" ({player_position}" + f", {player_points:.0f} pts" + f", {player_price:.1f} mill)")
    
    predicted_points = value(lp_prob.objective)
    predicted_points_per_player = value(lp_prob.objective) / (sum(form) + sum(sub))
    print(f"points: {predicted_points:.02f}")
    print(f"points/player: {predicted_points_per_player:.02f}")
    print()

(1, 3, 5, 2)
vincenzo grifo (midfield, 182 pts, 4.5 mill)
ridle baku (midfield, 107 pts, 2.4 mill)
filip kostic (midfield, 163 pts, 3.8 mill)
timo horn (goal, 123 pts, 1.0 mill)
marvin friedrich (defense, 107 pts, 2.5 mill)
karim onisiwo (attack, 108 pts, 2.6 mill)
andrej kramaric (attack, 184 pts, 4.8 mill)
robin knoche (defense, 100 pts, 2.0 mill)
robert andrich (midfield, 118 pts, 2.8 mill)
rafal gikiewicz (goal, 187 pts, 2.8 mill)
jonas hector (defense, 121 pts, 2.8 mill)
maximilian arnold (midfield, 134 pts, 2.6 mill)
christian günter (defense, 172 pts, 3.4 mill)
points: 1806.77
points/player: 138.98

(1, 4, 5, 1)
vincenzo grifo (midfield, 182 pts, 4.5 mill)
ridle baku (midfield, 107 pts, 2.4 mill)
filip kostic (midfield, 163 pts, 3.8 mill)
timo horn (goal, 123 pts, 1.0 mill)
marvin friedrich (defense, 107 pts, 2.5 mill)
karim onisiwo (attack, 108 pts, 2.6 mill)
andrej kramaric (attack, 184 pts, 4.8 mill)
robin knoche (defense, 100 pts, 2.0 mill)
robert andrich (midfield, 118 pts,